# Exercise 06: Geospatial Indexing with Python

**GIS 321: Principles of Programming for GIScience**  
**Instructor: Dr. Sergio Rey**

To run this notebook, you will need to download this [countries.geojson](https://github.com/datasets/geo-countries/tree/master/data) file (24MB) and place it in the same directory as this notebook.

1. Create a new class called Feature. 
2. Add a minimum bounding rectangle (MBR) attribute to your Feature class
3. Add a method to determine whether a point (lon, lat) is contained in the Feature's MBR.

In [61]:
import math
class Feature(object): 
    
    # Functions in classes are called "methods"
    # All methods must have an initial argument of "self"
    # Not used when calling, but it's how the function can access the 
    #  instantiated object's data.
    # The __init__ method is a special function that is run when an
    #  object is created
    def __init__(self, featureDict):
        self.geometry = featureDict["geometry"]
        self.properties = featureDict["properties"]
        self.airport_list = []
        
        # Minimum bounding rectangle is stored as 
        # [[upperleft x, upperleft y], [lowerright x, lowerright y]]
        self.mbr = [[math.inf,-math.inf],[-math.inf,math.inf]]
        if self.geometry["type"] == "Polygon":
            self.mbr = self.getPolygonMBR(self.geometry["coordinates"])
        elif self.geometry["type"] == "MultiPolygon":
            self.mbr = self.getMultiPolygonMBR(self.geometry["coordinates"])
        else:
            print("ERROR: Invalid geometry type '%s'"%self.geometry["type"])
        #
    #
    
    def isPointInMBR(self, point):
        """
        Takes point as (x,y) pair
        """
        return point[0] > self.mbr[0][0] and point[0] < self.mbr[1][0] and point[1] < self.mbr[0][1] and point[1] > self.mbr[1][1] 
    #
    
    def getPolygonMBR(self, polygon_coords):
        mbr = [[math.inf,-math.inf],[-math.inf,math.inf]]
        for point in polygon_coords[0]: # Only check the bounds of the first (outer) ring
            if point[0] < mbr[0][0]:
               mbr[0][0] = point[0]
            if point[0] > mbr[1][0]:
               mbr[1][0] = point[0]
            if point[1] > mbr[0][1]:
               mbr[0][1] = point[1]
            if point[1] < mbr[1][1]:
               mbr[1][1] = point[1]
        return mbr
    #
    
    def getMultiPolygonMBR(self, multipolygon_coords):
        mbr = [[math.inf,-math.inf],[-math.inf,math.inf]]
        for polygon in multipolygon_coords:
            tmbr = self.getPolygonMBR(polygon)
            # Check if any coordinate in the polygon's MBR extends the existing MBR
            if tmbr[0][0] <  mbr[0][0]:
                mbr[0][0] = tmbr[0][0]
            if tmbr[1][0] >  mbr[1][0]:
                mbr[1][0] = tmbr[1][0]
            if tmbr[0][1] >  mbr[0][1]:
                mbr[0][1] = tmbr[0][1]
            if tmbr[1][1] <  mbr[1][1]:
                mbr[1][1] = tmbr[1][1]
        return mbr
    #
#


1. For each feature in the countries file create an instance of your Feature class.

In [62]:
import json
f = "countries.geojson"
with open(f, 'r') as infile:
    g = json.load(infile)
g.keys()
# g is dictionary, which uses the GeoJSON format

# Convert all the GeoJSON features to Feature objects
feature_list = []
for f in g["features"]:
    feature_list.append(Feature(f))

Canada



#4. For each feature add a new attribute that lists the airports contained in its MBR.


In [63]:
# Load airports
import points
path = '../exercise04/airports.csv'
header, data = points.read_airport_csv(path)

In [64]:
# For each country, go through the airport list.
# If the airport point is in the country's MBR, add its ID to the airport_list
lat_idx = header.index("latitude")
lon_idx = header.index("longitude")
id_idx = header.index("airport_id")

for country in feature_list:
    for airport in data:
        if country.isPointInMBR((airport[lon_idx], airport[lat_idx])):
            country.airport_list.append(airport[id_idx])

In [84]:
#test: Country 39 is Canada, and most of its airports should be Canada
# 15 (Australia) is an even better match
country_id = 39

# print(g["features"][country_id]["properties"]["ADMIN"])
print("Country %d is %s.  Here are the countries in its airport list:"%(country_id, feature_list[country_id].properties["ADMIN"]))
airport_country_list = {}
for apt_id in feature_list[country_id].airport_list:
    country = points.get_airport_by_id(header, data, apt_id)[3]
    if country in airport_country_list:
        airport_country_list[country] += 1
    else:
        airport_country_list[country] = 1
print(airport_country_list)

Country 39 is Canada.  Here are the countries in its airport list:
{'Saint Pierre and Miquelon': 2, 'Canada': 382, 'United States': 327, 'Greenland': 9}
